# Time series analysis

In de gebruikte datasets tot nu toe is er verondersteld dat er geen correlatie is tussen de verschillende datapunten.
Dit is echter vaak niet het geval.
Bijvoorbeeld bij het voorspellen van beurskoersen of elektriciteitsverbruik.
In deze situaties zijn de datapunten geordend op basis van de tijd, dit wordt ook een tijdreeks of time-series genoemd, en zijn voorspelling sterk afhankelijk van de meest recente datapunten.
Meer informatie over time series analysis kan je ook [hier](https://towardsdatascience.com/time-series-analysis-using-pandas-in-python-f726d87a97d8) en [hier](https://towardsdatascience.com/time-series-modeling-using-scikit-pandas-and-numpy-682e3b8db8d1) vinden.

Waarom is het nu belangrijk om extra aandacht te besteden aan tijdreeksen?
Hoe zou de toekomst voorspeld kunnen worden van de volgende tijdreeks?
Een eenvoudig idee zou gewoon zijn om de tijdreeks met 1 dag te verschuiven.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
url='https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
data = pd.read_csv(url, index_col=0, parse_dates=True)
cols_plot = ['Consumption', 'Solar', 'Wind']
axes = data[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11, 9), subplots=True)
for ax in axes:
    ax.set_ylabel('Daily Totals (GWh)')

In [ ]:
display(data.tail())
data.index

Bij het bestuderen valt het op dat de frequentie waarin de data opgenomen is duidelijk 1 rij per dag is.
Deze frequentie is echter niet gezet in de index (freq = None).
Dit kan gecorrigeerd worden als volgt

In [ ]:
data = data.asfreq('D', method = 'ffill')

Andere veel gebruikte frequenties die gebruikt kunnen worden zijn
* "H": per uur
* "D": per dag
* "B": per werkdag
* "W": per week
* "M": per maand
* "Q": per kwartaal
* "Y": per jaar

De methode die hier gebruikt is, is forward fill.
Dit betekend dat als een rij Nan is, dat dan de waarde van de vorige rij gebruikt wordt.
Hiermee worden gaten in de dataset opgevuld.
In tijdreeksen is het namelijk niet aangewezen op vast waarden zoals het gemiddelde of constanten te nemen omdat de waarden gecorreleerd zijn met de tijd. 

### Resampling

Doordat de index en de frequentie van de tijdreeks correct ingesteld is, is het eenvoudig om de tijdreeks op andere tijdsschalen te bekijken.
Dit wordt ook resampling genoemd en kan als volgt gedaan worden waar er bijvoorbeeld het gemiddelde van de hele week of maximum van de maand gekozen kan worden.

### Rolling Windows

Een nadeel van bovenstaande resampling methoden is dat het aantal datapunten sterk gereduceerd wordt.
Bij resampling werden de datapunten verdeeld in niet-overlappende groepen of bins.
Bij rolling windows definieren we een groep of bin voor elk datapunt.
De bins hierbij mogen wel overlappen.
Dit kan bijvoorbeeld gebruikt worden om de gegevens uit te middelen of om de trend van de data te bestuderen.

### Detrending

In bovenstaande figuur is het duidelijk dat de algemene elektriciteits consumptie redelijk stabiel is met twee dippen in 2009 en 2013.
Voor de trainingstijd en performantie van het model is het vaak aan te raden om de trend in een tijdreeks te verwijderen. 
Het verwijderen van de trendlijn zorgt ervoor dat de tijdreeks stationary wordt.
Dat wil zeggen dat de statistische eigenschappen zoals het gemiddelde en variantie constant blijft door de tijd.

De methode om dit te doen wordt differencing genoemd. 
Hierbij wordt er een nieuwe tijdreeks berekend die het verschil weergeeft tussen tijdstip t en t-1.
De formule voor dit te doen is:

$x'(t) = x(t) - x(t-1)$

In sommige situaties kan het mogelijk zijn dat het eerste order verschil er niet in slaagt om de trendlijn te verwijderen.
Een oplossing hiervoor is om gebruik te maken van het tweede orde verschil wat kan berekend worden met onderstaande formule

$x'(t) = x(t) - 2* x(t-1) + x(t-2)$

### Seasonality of periodicity

Tijdseries die gebaseerd zijn op het menselijk leven (zoals elektriciteitsconsumptie of de seizoenen) gaan zichzelf vaak herhalen met bepaalde perioden (bvb elk jaar, week, dag).
In bovenstaande figuren viel het al reeds op dat de zowel een jaarlijkse als een wekelijkse interval heeft dat zich herhaalt.

### Autocorrelation

Dit is een techniek om periodicity te gaan analyseren door de correlatie van de tijdreeks met een verschoven tijdreeks te gaan tekenen voor verschillende tijdsverschuivingen.

Intuitief wil dit het volgende zeggen: je verschuift de tijdseries met x aantal dagen/weken/maanden en berekent het verschil tussen de twee tijdseries.
Indien de tijdreeks een periodiciteit heeft van x dagen/weken/maanden dan zal dit verschil heel klein zijn of de correlatie heel hoog.
Door dit voor verschillende verschuivingen te doen kunnen de verschillende periodiciteiten gedetecteerd worden.

De resulterende plot ziet er als volgt uit:

Hoe moet deze figuur geinterpreteerd worden?
* de x-as stelt de verschuiving van de data voor
* de y-as geeft de correlatie tussen de verschoven tijdreeks en de originele weer. Wanneer dit een piek vertoond, dan kan je stellen dat de tijdreeks zich na zoveel dagen herhaalt.
* De horizontale lijnen geven een indicatie van welke pieken statistisch relevant zijn. Elke piek die boven de gestreepte lijnen komt geeft aanlijding tot een herhaling van de tijdreeks.

Bovenstaande auto-correlation plot werd getekend voor de originele data.
We kunnen dit ook eens doen voor de tijdreeks waar de trend in de data verwijderd is. 
In deze plot is de periodiciteit sterker zichtbaar wat het belang van detrending weergeeft.

## Modelleren van tijdreeksen

Er zijn een aantal geavanceerde packages beschikbaar voor het werken met tijdreeksen zoals ARIMA of Prophet (van Facebook).
Het is echter perfect mogelijk om de verschillende reeds geziene ML-technieken toe te passen op tijdreeksen.
Er zijn wel een paar belangrijke aandachtspunten om ervoor te zorgen dat je de orde van de tijdspunten respecteert.
Om dit aan te tonen gaan we proberen om het totale verbruik te voorspellen.
De dataset die beschikbaar is bevat data van 2006 tot en met 2017.
We kunnen bvb de eerste 11 jaar gebruiken voor het model te trainen en het laatste jaar te gebruiken als testdata om de performantie van het model te bepalen.

## Poging 1

Een eerste probleem bij het voorspellen van een tijdreeks is dat er een sterke mogelijkheid is dat het model steeds de laatste geziene waarde leert voorspellen.
Dit komt omdat er typisch bij tijdreeksen een grote kans is dat het verschil op 1 dag klein is.
Zo is de eenvoudigste manier om het weer van morgen te voorspellen te zeggen dat het hetzelfde gaat zijn als vandaag.

Dit kan je al wat zien op onderstaande figuur waar de blauwe lijn precies een beetje voorloopt op de orange lijn (namelijk 1 dag)
Het verschil in grootteordes is hier niet zo belangrijk. Dit komt vooral doordat er doorheen de jaren een stijgend gebruik is wat hier ook niet correct mee gemodelleerd kan worden.

Om bovenstaande probleem op te lossen kan het handig zijn om eerder te werken met de verschillen tussen opeenvolgende dagen dan met de echte waarden.
De simpelste features die gebruikt kunnen worden om het verbruik van vandaag te voorspellen zijn het verbruik van gisteren en het verschil van het verbruik van gisteren en eergisteren.
Het dataframe met deze data kan als volgt bepaald worden.

Merk op dat de eerste twee dagen verdwenen zijn.
Dit komt omdat er twee dagen nodig zijn om de waarden van de twee features te bepalen.
Aangezien dit niet kan voor de eerste twee dagen resulteert dit in NaN waarden waardoor de rijen wegvallen.

Nu kunnen we de beschikbare data verdelen in test en trainingsdata.

#### Cross validation

Bij het werken met tijdreeksen kan er niet gebruik gemaakt worden van de standaard cross-validation technieken omdat deze de data willekeurig selecteert.
Hierdoor wordt de volgorde van de data niet gerespecteerd wat voor problemen zorgt.
Een betere strategie voor cross-validation te gebruiken dan de standaard K-fold is de Forward Chaining techniek.
In het geval van 3 folds in een dataset van 4 jaar zien de train en validatie sets er uit als volgt:
* fold 1: training [jaar 1], validation [jaar 2]
* fold 2: training [jaar 1 en jaar 2], validation [jaar 3]
* fold 3: training [jaar 1 tot jaar 3], validation [jaar 4]

Om deze folds te bepalen van de datasets bied sklearn de TimeSeriesSplit klasse aan. Hieronder berekenen we 10 folds (omdat de trainingsdata uit 11 jaar bestaat)

#### Modellen trainen

Nu is de data beschikbaar om de verschillende ML-learning technieken op de data los te laten.

Hieronder gaan we verder enkel gebruik maken van de Random Forest techniek.
Voor deze techniek gaan we met een grid search op zoek naar het model dat het beste het verbruik kan voorspellen.

## Poging 2

Tot nu toe is enkel het verbruik van de vorige dag in rekening gebracht.
We kunnen echter ook verder terug gaan het verleden, bvb tot 2 dagen.
Dit geeft het volgende resultaat.

## Poging 3

In de vorige twee pogingen is enkel gebruik gemaakt van informatie over het verbruik.
Het is ook mogelijk om de productie van zonnepanelen toe te voegen aan de dataset.
Dit resulteert in het volgende model.

# Poging 4

In het begin is er vastgesteld geweest dat de periodiciteit van de dataset een week is of 7 dagen.
Een ander idee voor een extra feature is om minstens het verbruik van van een week geleden toe te voegen.

## Belang van de features

Welke feature in de dataset levert nu het meest informatie op om het target te voorspellen?
Dit kan bepaald worden door het opstellen van een variable important plot.
In deze plot is het duidelijk te zien dat de informatie van de week ervoor veel belangrijker is om het verbruik te voorspellen dat de waarde van de dag ervoor.